In [1]:
import tensorflow as tf
import pandas as pd
import os
import time


/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


###para la inferencia podemos o espera un intervalod e tiempo para crear un eventcount_vector o sera que creamos una pequena memoria con cierta cantidad de lieas de logs hacia atras y vamos creando vectores y probando.

Creo que no tiene sentido hacer inferencia de linea en linea
 
- TODO: pensar y validar en codigo como hacer la inferencia de vainas nuevas
 

- definir cuales son los indicadores que se mostraran a dayco, tiempo de procesamiento, memoria, eficiencia delproceso

- justificar con argumentos porque el proceso funcionara con la data dayco igual que con la data de prueba

- como hacer en tiempo de inferencia, podemos simular que un archivo se va llenando de logs???revisar como cambia un archivo de logs en el logparser???

In [3]:
'''
!apt-get update
!apt-get install -y wget bzip2
!apt-get install -y gcc perl git
!rm -rf /var/lib/apt/lists/*

!cd /
!mkdir anaconda
!cd anaconda
!wget https://repo.anaconda.com/archive/Anaconda2-5.2.0-Linux-x86_64.sh
!bash Anaconda2-5.2.0-Linux-x86_64.sh \n\n\n\n\n\n\n\n\n
!source ~/.bashrc
!cd ..
!rm -r anaconda 
'''

'\n!apt-get update\n!apt-get install -y wget bzip2\n!apt-get install -y gcc perl git\n!rm -rf /var/lib/apt/lists/*\n\n!cd /\n!mkdir anaconda\n!cd anaconda\n!wget https://repo.anaconda.com/archive/Anaconda2-5.2.0-Linux-x86_64.sh\n!bash Anaconda2-5.2.0-Linux-x86_64.sh \n\n\n\n\n\n\n\n\n\n!source ~/.bashrc\n!cd ..\n!rm -r anaconda \n'

## clonamos loglizer

In [0]:
#!git clone https://github.com/elieser1101/loglizer.git /loglizer/

Cloning into '/loglizer'...
remote: Enumerating objects: 156, done.
remote: Counting objects: 100% (156/156), done.
remote: Compressing objects: 100% (99/99), done.
remote: Total 414 (delta 90), reused 107 (delta 43), pack-reused 258
Receiving objects: 100% (414/414), 2.84 MiB | 14.25 MiB/s, done.
Resolving deltas: 100% (235/235), done.


In [2]:
#os.chdir('/loglizer')
#!git pull origin master

remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 4 (delta 3), reused 4 (delta 3), pack-reused 0
Unpacking objects: 100% (4/4), done.
From https://github.com/elieser1101/loglizer
 * branch            master     -> FETCH_HEAD
   4566141..6ff9d52  master     -> origin/master
Updating 4566141..6ff9d52
Fast-forward
 pipeline/pipeline.py | 2 ++
 1 file changed, 2 insertions(+)


## iniciamos submodulo de git

In [0]:
#os.chdir('/loglizer')
#!git submodule update --init

Submodule 'logparser' (https://github.com/elieser1101/logparser.git) registered for path 'logparser'
Cloning into '/loglizer/logparser'...
Submodule path 'logparser': checked out '7352c48e89f8a3984ccf82f2c7741c3ae8f1aeb4'


In [4]:
!pwd
!ls

/home/us1/git/loglizer
11_2_pipeline_deepia_intentto_online.ipynb  demo_hdfs	 logparser  rocore1.log
dayco_log.log_structured.csv		    img		 models     rocore2.log
dayco_log.log_templates.csv		    __init__.py  pipeline   utils
demo_bgl				    LICENSE	 README.md


## importamos sys y agregamos algunas rutas necesarias para el logparser al path de python

In [6]:
import sys
#TODO: esto se debe incluir en repo, ver si hay una manera correcta o si es haciendo append del path!
#os.chdir('/loglizer')
repo_path = '/home/us1/git/loglizer'
sys.path.append(repo_path + '/logparser/LogSig')#logsig __init__.py
sys.path.append(repo_path + '/logparser/logparser/LenMa/')#for lenma __init__.py
sys.path.append(repo_path + '/logparser/logparser/LenMa/templateminer')#for lenma
from pipeline.pipeline import Pipeline
from pipeline.pipeline import Parser
from pipeline.pipeline import Loglizer
#os.chdir('/loglizer')

## mining invariants

### hack para probar el loglizer en pipeline,
luego de instanciar pipeline, cambiamos los atributos, para que apunten al dataset con el que funciona mining invariants, y creamos una nueva instancia de loglizer actualmente el `loglizer_input_dir` apunta a la salida del parser que es correcto, pero hasta que no se tenga el MiningInvariants adaptado a esa entrada se rompera
<br><br>
###NOTA: se puede jugar y obtener cualquier intancia de parser modificando los atributos de pipeline y haciendo get_parser, lo mismo para los loglizers

### visualizamos logs dayco

In [8]:
!head {repo_path}/rocore1.log
print('******')
!head {repo_path}/rocore2.log

Oct  4 14:30:04 200.74.216.1 3290: 009042: Oct  4 14:31:07.046 CCS: %SYS-6-LOGGINGHOST_STARTSTOP: Logging to host 192.168.1.140 port 0 CLI Request Triggered
Oct  4 14:30:05 200.74.216.1 3291: 009043: Oct  4 14:31:08.046 CCS: %SYS-6-LOGGINGHOST_STARTSTOP: Logging to host 192.168.1.140 port 514 started - CLI initiated
Oct  4 14:30:39 200.74.216.1 3292: 009044: Oct  4 14:31:41.731 CCS: %SYS-5-CONFIG_I: Configured from console by admbautistay on vty0 (200.74.215.2)
Oct  4 15:02:04 200.74.216.1 3293: 009052: Oct  4 15:03:07.365 CCS: %SYS-6-TTY_EXPIRE_TIMER: (exec timer expired, tty 1 (200.74.215.2)), user admbautistay
Oct  4 15:02:04 200.74.216.1 3294: 009053: Oct  4 15:03:07.365 CCS: %SYS-6-LOGOUT: User admbautistay has exited tty session 1(200.74.215.2)
Oct  4 21:01:43 200.74.216.1 3295: 009069: Oct  4 21:02:46.664 CCS: %SEC_LOGIN-5-LOGIN_SUCCESS: Login Success [user: admcod] [Source: 200.74.215.2] [localport: 22] at 21:02:46 CCS Thu Oct 4 2018
Oct  4 21:01:46 200.74.216.1 3296: 009070: O

# Pipeline
## intentemos correr pipeline


### nota importante, es deinteres seleccionar con cuidado el parser pues, de la cantidad de eventos que este descubra dependera el gasto computacional del mining invarina


### seleccionaamos la cantidade de lineas con la que trabajaremos

In [10]:
!cp {repo_path}/rocore2.log {repo_path}/dayco_log.log

!wc -l {repo_path}/dayco_log.log
!ls -sh {repo_path}/dayco_log.log

234 /home/us1/git/loglizer/dayco_log.log
36K /home/us1/git/loglizer/dayco_log.log


In [11]:
#solo par auna pruea
!head -100 {repo_path}/rocore2.log > {repo_path}/dayco_log.log
!wc -l {repo_path}/dayco_log.log


100 /home/us1/git/loglizer/dayco_log.log


### NOTA: es importante que la ultima linea del archivo inicial con el que trabajeroms no contenga el salto de linea
nosotros simplemente estamos copiando la ultima linea sin el salto de linea
### NOTA:cuando se utilice un dataset completo tener cuidado de no darle el tratamiento que sigue

In [12]:

file = open(repo_path+'/dayco_log.log', 'r')
lines = file.readlines()
file.close()
file = open(repo_path+'/dayco_log.log', 'a')
file.write(lines[-1][:-1])
file.close()
file = open(repo_path+'/dayco_log.log', 'r')
lines = file.readlines()
file.close()

In [13]:
print(len(lines))
print(lines[-2:])
!wc -l {repo_path}/dayco_log.log


101
['Oct 10 17:42:45 200.74.216.13 406: 001043: Oct 10 17:43:53.549 CCS: %SYS-6-LOGOUT: User admcod has exited tty session 1(200.74.224.3)\n', 'Oct 10 17:42:45 200.74.216.13 406: 001043: Oct 10 17:43:53.549 CCS: %SYS-6-LOGOUT: User admcod has exited tty session 1(200.74.224.3)']
100 /home/us1/git/loglizer/dayco_log.log


In [14]:
input_dir  = repo_path +'/'  # The input directory of log file
output_dir = repo_path + '/'  # The output directory of parsing results
log_file   = 'dayco_log.log'  # The input log file name
#log_format = '<Label> <Timestamp> <Date> <Node> <Time> <NodeRepeat> <Type> <Component> <Level> <Content>'#BGL
log_format = '<smonth> <sday> <shour> <ip> <id> <id2> <month> <day> <hour> <city> <type> <Content>'#dayco/rsyslog

pipeline = Pipeline(parser_algorithm='drain', input_dir = input_dir, parser_output_dir = output_dir, log_file = log_file,
parser_regex = log_format, feature_extractor='fixed_window', log_analizer_algorithm='mining_invariants',
data_type='time_based')

### en time_windows el feature extracto guarda una copia de lso indices de inicio y final de cada sliding window, por eso debemos borrar ese directorio cuando vamos a probar con archivos de diferentes tamanos(si se mantien la misma configuracion de swliding window y step size)

In [0]:
!rm /dayco_log.log_structured.csv
!rm /dayco_log.log_templates.csv
!rm /dayco_log.logTemplateMap.csv
!rm -r /time_windows

### Logica Online

parsing
- vamos parseando cada evento que entra???
- esperamos que entre un minimo de logs???
- parsemos cada cierto tiempo???
<br> ##############<br>

que hacemos con el event count matrix??
- cada cuanto la actualizamos??
- la guardamos en csv?
- validamos la longitud y en funcion de si cmabio iniciamos otros procesos?
<br> ##############<br>

calculo de invarianes(tomar ne cuent que en general se tarda bastate esto)
- cada cuanto debemos correr encontrar invariantes??
- si sabemos que es poca data podemos correrlo cada vez que ingresa un log nuevo??
- es practico buscar invariantes con mucha periodicidad??
<br> ##############<br>

debemos hacer inferencia con log que vayan entrando
- esperamos que se tengan un minimo de eventos para armar un event count vector??
- esperamos x unidades de tiempo??? 
- o con cad log probamoss??

++++++++++++++++++++++++++++++++++++++++<br>
se me ocurre crear un metodo fisrt_time_trining, que ejecute parsing y genere una primer event_count_matrix y encuentre los primeros invariantes

pedir que se evaluen todos estos enventos

crear un metodo depia daemon, 
1.
que evalue si el logfile.log incremento su tamano o si cambio la fecha del ultimo registro, calcular cual es la diferencia de lineas(guardar cual fue la ultima linea que se parseo), ejecutar parser sobre diferencia del log
2.
sobre la diferencia del log agregar nuevo event vector, tomar en cuenta el echo de que se utiliza sliding windows, como incluyo el evento en los logs anteriores?????? se puede/tiene que actualizar la matriz???o mejor se calcula una nueva???
3. 
definimos un trshol de cantidad de logs para reiniciar el calculo de invariantes??
o mas bien el treshold debe ser temporal???
POR ULTIMO, se puede evaluar la posibilidad de ir guardando en csv un registro de invariantes???es tendria sentido????(creo que no porque me parece que los invariantes calculados solo con nuevos vectores romperan cundo se comparen con vectores viejos, ahor los viejos romperan con los nuevos??)
4.
partiendo de 1 y 2 deberiamos tener los nuevos vectores y simplemente podemos hacer una infeencia con los invriantes que se tengan para el momento1234

## corremos el proceso de generar event_count_matrix, encontrar invariantes y detectar anomalias de data original
el pipeline es capaz de correr sin etiquetas, con los metodos deepia


In [29]:
!rm {repo_path}/time_windows/sliding_3h_1h.csv

In [30]:
#loglizer params for BGL
para = {
'path':repo_path + '/',                 # directory for input data
'log_file_name':'dayco_log.log',           # filename for log data file
'log_event_mapping':'dayco_log.logTemplateMap.csv',   # filename for log-event mapping. A list of event index, where each row represents a log
'save_path': './time_windows/',             # dir for saving sliding window data files to avoid splitting
#'select_column':[0,4],                      # select the corresponding columns (label and time) in the raw log file
'select_column':[0,1,2],                      # select the corresponding columns (label and time) in the raw log file, en caso de depia no nos intersa la columna label
'window_size':3, #3                           # time window (unit: hour)
'step_size': 1,                             # step size (unit: hour)
'epsilon':2.0,                              # threshold for the step of estimating invariant space
'threshold':0.97,                           # percentage of vector Xj in matrix satisfies the condition that |Xj*Vi|<epsilon
'scale_list':[1,2,3],					    # list used to sacle the theta of float into integer
'stop_invar_num':3                          # stop if the invariant length is larger than stop_invar_num. None if not given
}
start = time.time()
#event_count_matrix = pipeline.log_analizer.get_event_count_matrix(para)
#invar_dict = pipeline.log_analizer.find_invariants(para, event_count_matrix)
#predictions, anomalies = pipeline.log_analizer.get_anomalies(para, event_count_matrix, invar_dict)
pipeline.initial_go(para)
end = time.time()
print('total execution time in seconds?',end - start)

Parsing file: /home/us1/git/loglizer/dayco_log.log
Processed 100.0% of log lines.
Parsing done. [Time taken: 0:00:00.039375]
The raw data shape is (101, 1) and label shape is (101, 1)
there are 171 instances (sliding windows) in this dataset

There are 24 log events
(171, 24)
SVD decomposition results (U,S,V) size:  (24, 24) (24,) (24, 24)
The size of invariants should be 23
the remaining features are:  [[0], [1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [12], [13], [14], [15], [16], [17], [18], [19], [20], [21], [22], [23]]
selected matrix columns are [0, 1]
selected matrix columns are [0, 2]
selected matrix columns are [0, 3]
0 exits and discard the following vector: 
selected matrix columns are [0, 4]
0 exits and discard the following vector: 
selected matrix columns are [0, 5]
0 exits and discard the following vector: 
selected matrix columns are [0, 6]
selected matrix columns are [0, 7]
0 exits and discard the following vector: 
selected matrix columns are [0, 8]
select

0 exits and discard the following vector: 
selected matrix columns are [5, 18]
0 exits and discard the following vector: 
selected matrix columns are [5, 19]
0 exits and discard the following vector: 
selected matrix columns are [5, 20]
0 exits and discard the following vector: 
selected matrix columns are [5, 21]
0 exits and discard the following vector: 
selected matrix columns are [5, 22]
0 exits and discard the following vector: 
selected matrix columns are [5, 23]
0 exits and discard the following vector: 
selected matrix columns are [6, 7]
0 exits and discard the following vector: 
selected matrix columns are [6, 8]
0 exits and discard the following vector: 
selected matrix columns are [6, 9]
0 exits and discard the following vector: 
selected matrix columns are [6, 10]
0 exits and discard the following vector: 
selected matrix columns are [6, 11]
0 exits and discard the following vector: 
selected matrix columns are [6, 12]
0 exits and discard the following vector: 
selected mat

selected matrix columns are [0, 1, 2]
selected matrix columns are [0, 1, 3]
0 exits and discard the following vector: 
selected matrix columns are [0, 1, 4]
0 exits and discard the following vector: 
selected matrix columns are [0, 1, 5]
0 exits and discard the following vector: 
selected matrix columns are [0, 1, 6]
selected matrix columns are [0, 1, 7]
0 exits and discard the following vector: 
selected matrix columns are [0, 1, 8]
selected matrix columns are [0, 1, 9]
selected matrix columns are [0, 1, 10]
selected matrix columns are [0, 1, 11]
selected matrix columns are [0, 1, 12]
selected matrix columns are [0, 1, 13]
selected matrix columns are [0, 1, 14]
selected matrix columns are [0, 1, 15]
selected matrix columns are [0, 1, 16]
selected matrix columns are [0, 1, 17]
selected matrix columns are [0, 1, 18]
selected matrix columns are [0, 1, 19]
selected matrix columns are [0, 1, 20]
selected matrix columns are [0, 1, 21]
0 exits and discard the following vector: 
selected matr

0 exits and discard the following vector: 
selected matrix columns are [2, 3, 11]
0 exits and discard the following vector: 
selected matrix columns are [2, 3, 12]
0 exits and discard the following vector: 
selected matrix columns are [2, 3, 13]
0 exits and discard the following vector: 
selected matrix columns are [2, 3, 14]
0 exits and discard the following vector: 
selected matrix columns are [2, 3, 15]
0 exits and discard the following vector: 
selected matrix columns are [2, 3, 16]
0 exits and discard the following vector: 
selected matrix columns are [2, 3, 17]
0 exits and discard the following vector: 
selected matrix columns are [2, 3, 18]
0 exits and discard the following vector: 
selected matrix columns are [2, 3, 19]
0 exits and discard the following vector: 
selected matrix columns are [2, 3, 20]
0 exits and discard the following vector: 
selected matrix columns are [2, 3, 21]
0 exits and discard the following vector: 
selected matrix columns are [2, 3, 22]
0 exits and disc

0 exits and discard the following vector: 
selected matrix columns are [5, 11, 17]
0 exits and discard the following vector: 
selected matrix columns are [5, 11, 18]
0 exits and discard the following vector: 
selected matrix columns are [5, 11, 19]
0 exits and discard the following vector: 
selected matrix columns are [5, 11, 21]
0 exits and discard the following vector: 
selected matrix columns are [5, 11, 22]
0 exits and discard the following vector: 
selected matrix columns are [5, 11, 23]
0 exits and discard the following vector: 
selected matrix columns are [5, 13, 15]
0 exits and discard the following vector: 
selected matrix columns are [5, 13, 16]
0 exits and discard the following vector: 
selected matrix columns are [5, 13, 17]
0 exits and discard the following vector: 
selected matrix columns are [5, 13, 18]
0 exits and discard the following vector: 
selected matrix columns are [5, 13, 19]
0 exits and discard the following vector: 
selected matrix columns are [5, 13, 20]
0 ex

65  Oct  9  14:17:57  200.74.216.13  372:  000973:  Oct  9  14:19:03.110  CCS:  %BGP-5-NBR_RESET:  Neighbor 190.52.105.93 reset (BGP Notification...  29709cb4  Neighbor <*> reset (BGP Notification <*>
*******8anomaly*****************
index, row 145 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1.]
cols [8, 12] [-6.0, 1.0]

 invariant [8, 12]
>>> received from neighbor 200.16.71.1 6/6 (Other Configuration Change) 0 bytes
start, end 77 81
len de eventos en window 0
>>> neighbor <*> Up
start, end 77 81
len de eventos en window 1
81  Oct  9  16:32:54  200.74.216.13  387:  000992:  Oct  9  16:34:01.507  CCS:  %BGP-5-ADJCHANGE:  neighbor 200.74.211.254 Up  9fc2ddbe  neighbor <*> Up
total execution time in seconds? 1.093130350112915


In [31]:
pipeline.last_log_lines_len

101

In [32]:
print(len(pipeline.invar_dict))
pipeline.invar_dict

19


{frozenset({4, 5}): array([ 1., -1.]),
 frozenset({8, 9}): array([-2.,  1.]),
 frozenset({8, 10}): array([-2.,  1.]),
 frozenset({8, 11}): array([-2.,  1.]),
 frozenset({8, 12}): array([-6.,  1.]),
 frozenset({8, 13}): array([ 1., -1.]),
 frozenset({8, 14}): array([ 1., -1.]),
 frozenset({9, 20}): array([ 1., -2.]),
 frozenset({10, 20}): array([ 1., -2.]),
 frozenset({11, 20}): array([ 1., -2.]),
 frozenset({12, 15}): array([ 1., -1.]),
 frozenset({12, 16}): array([ 1., -1.]),
 frozenset({12, 17}): array([ 1., -1.]),
 frozenset({12, 18}): array([ 1., -3.]),
 frozenset({12, 19}): array([ 1., -3.]),
 frozenset({12, 20}): array([ 1., -6.]),
 frozenset({12, 21}): array([ 1., -6.]),
 frozenset({12, 22}): array([ 1., -6.]),
 frozenset({12, 23}): array([ 1., -6.])}

In [33]:
pipeline.event_count_matrix.shape

(171, 24)

In [34]:
print(len(pipeline.anomalies))
for row_anomaly in pipeline.anomalies:
  print('##########################row anomalies')
  for event_anomaly in row_anomaly.anomaly_events:
    print('\n')
    print(event_anomaly.event_template)
    print(event_anomaly.row_index, event_anomaly.row)
    print(event_anomaly.invar_cols)
    print(event_anomaly.invariants)
    print(event_anomaly.start_window_index,event_anomaly.end_window_index)
    print(event_anomaly.matching_log_list)
    print(event_anomaly.log_lines_list)

4
##########################row anomalies


received from neighbor 200.16.71.1 6/6 (Other Configuration Change) 0 bytes
140 [1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[8, 9]
[-2.0, 1.0]
42 50
['43  Oct  9  11:24:28  200.74.216.13  349:  000947:  Oct  9  11:25:29.942  CCS:  %BGP-3-NOTIFICATION:  received from neighbor 200.16.71.1 6/6 (Other ...  848a25c6  received from neighbor 200.16.71.1 6/6 (Other ...']
['43']


Neighbor <*> reset (BGP Notification <*>
140 [1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[8, 9]
[-2.0, 1.0]
42 50
['44  Oct  9  11:24:28  200.74.216.13  350:  000948:  Oct  9  11:25:29.942  CCS:  %BGP-5-NBR_RESET:  Neighbor 200.16.71.1 reset (BGP Notification r...  29709cb4  Neighbor <*> reset (BGP Notification <*>']
['44']
##########################row anomalies


received from neighbor 200.16.71.1 6/6 (Other Configuration Change) 0 bytes
142 [2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 0. 0. 2. 2. 2. 1. 1. 0. 0. 0. 0.]

In [35]:
data_dft = pd.read_csv(repo_path + '/time_windows/sliding_3h_1h.csv', delimiter=r',', header=None) #, parse_dates = [1], date_parser=dateparse)
#data_dft = data_dft[0:100]
data_dft['diff'] = data_dft[1] - data_dft[0]
print(len(data_dft))
data_dft

171


,0,1,diff
0,0,3,3
1,3,3,0
2,3,3,0
3,3,3,0
4,3,3,0
5,3,5,2
6,5,5,0
7,5,5,0
8,5,5,0
9,5,5,0


### simulamos un cambio en el log, y al ejecutar pipeline.deepslash se valida el cambio, parsea,  genremaos eventcount matrix y hacemos inferenciacon los mismo invariantes ue se calcularon primero

vemos que priemro eran 27 eventos y luego 29
tambien vemos que primeor se detectan 10 anomalias y uego 14

In [36]:
!head -150 {repo_path}/rocore2.log > {repo_path}/dayco_log.log

!wc -l {repo_path}/dayco_log.log
!ls -sh {repo_path}/dayco_log.log

file = open(repo_path + '/dayco_log.log', 'r')
lines = file.readlines()
file.close()
file = open(repo_path + '/dayco_log.log', 'a')
file.write(lines[-1][:-1])
file.close()
file = open(repo_path + '/dayco_log.log', 'r')
lines = file.readlines()
file.close()

150 /home/us1/git/loglizer/dayco_log.log
24K /home/us1/git/loglizer/dayco_log.log


In [37]:
pipeline.deepslash(para)

old, new 101 151
Parsing file: /home/us1/git/loglizer/dayco_log.log
Processed 100.0% of log lines.
Parsing done. [Time taken: 0:00:00.040156]
The raw data shape is (151, 1) and label shape is (151, 1)
there are 250 instances (sliding windows) in this dataset

There are 27 log events
log template path is /home/us1/git/loglizer/dayco_log.log_templates.csv
the mined 19 invariants are: {frozenset({4, 5}): array([ 1., -1.]), frozenset({8, 9}): array([-2.,  1.]), frozenset({8, 10}): array([-2.,  1.]), frozenset({8, 11}): array([-2.,  1.]), frozenset({8, 12}): array([-6.,  1.]), frozenset({8, 13}): array([ 1., -1.]), frozenset({8, 14}): array([ 1., -1.]), frozenset({9, 20}): array([ 1., -2.]), frozenset({10, 20}): array([ 1., -2.]), frozenset({11, 20}): array([ 1., -2.]), frozenset({12, 15}): array([ 1., -1.]), frozenset({16, 12}): array([ 1., -1.]), frozenset({17, 12}): array([ 1., -1.]), frozenset({18, 12}): array([ 1., -3.]), frozenset({19, 12}): array([ 1., -3.]), frozenset({12, 20}): arr

In [38]:
print(len(pipeline.anomalies))
for row_anomaly in pipeline.anomalies:
  print('##########################row anomalies')
  for event_anomaly in row_anomaly.anomaly_events:
    print('\n')
    print(event_anomaly.event_template)
    print(event_anomaly.row_index, event_anomaly.row)
    print(event_anomaly.invar_cols)
    print(event_anomaly.invariants)
    print(event_anomaly.start_window_index,event_anomaly.end_window_index)
    print(event_anomaly.matching_log_list)
    print(event_anomaly.log_lines_list)

7
##########################row anomalies


received from neighbor 200.16.71.1 6/6 (Other Configuration Change) 0 bytes
140 [1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.]
[8, 9]
[-2.0, 1.0]
42 50
['43  Oct  9  11:24:28  200.74.216.13  349:  000947:  Oct  9  11:25:29.942  CCS:  %BGP-3-NOTIFICATION:  received from neighbor 200.16.71.1 6/6 (Other ...  848a25c6  received from neighbor 200.16.71.1 6/6 (Other ...']
['43']


Neighbor <*> reset (BGP Notification <*>
140 [1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.]
[8, 9]
[-2.0, 1.0]
42 50
['44  Oct  9  11:24:28  200.74.216.13  350:  000948:  Oct  9  11:25:29.942  CCS:  %BGP-5-NBR_RESET:  Neighbor 200.16.71.1 reset (BGP Notification r...  29709cb4  Neighbor <*> reset (BGP Notification <*>']
['44']
##########################row anomalies


received from neighbor 200.16.71.1 6/6 (Other Configuration Change) 0 bytes
142 [2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 0. 0. 2. 2. 2

In [39]:
data_dft = pd.read_csv(repo_path + '/time_windows/sliding_3h_1h.csv', delimiter=r',', header=None) #, parse_dates = [1], date_parser=dateparse)
#data_dft = data_dft[0:100]
data_dft['diff'] = data_dft[1] - data_dft[0]
print(len(data_dft))
data_dft

250


,0,1,diff
0,0,3,3
1,3,3,0
2,3,3,0
3,3,3,0
4,3,3,0
5,3,5,2
6,5,5,0
7,5,5,0
8,5,5,0
9,5,5,0


In [40]:
pipeline.find_invariants(para)
print(len(pipeline.invar_dict))
pipeline.invar_dict

(250, 27)
SVD decomposition results (U,S,V) size:  (27, 27) (27,) (27, 27)
The size of invariants should be 26
the remaining features are:  [[0], [1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [12], [13], [14], [15], [16], [17], [18], [19], [20], [21], [22], [23], [24], [25], [26]]
selected matrix columns are [0, 1]
selected matrix columns are [0, 2]
selected matrix columns are [0, 3]
0 exits and discard the following vector: 
selected matrix columns are [0, 4]
0 exits and discard the following vector: 
selected matrix columns are [0, 5]
0 exits and discard the following vector: 
selected matrix columns are [0, 6]
selected matrix columns are [0, 7]
0 exits and discard the following vector: 
selected matrix columns are [0, 8]
selected matrix columns are [0, 9]
selected matrix columns are [0, 10]
selected matrix columns are [0, 11]
selected matrix columns are [0, 12]
selected matrix columns are [0, 13]
selected matrix columns are [0, 14]
selected matrix columns are [0, 15]
sele

{frozenset({4, 5}): array([-1.,  1.]),
 frozenset({4, 10}): array([-12.,   1.]),
 frozenset({4, 11}): array([-8.,  1.]),
 frozenset({4, 20}): array([-11.,   1.]),
 frozenset({4, 24}): array([ 1., -6.]),
 frozenset({4, 25}): array([ 1., -6.]),
 frozenset({4, 26}): array([ 1., -6.]),
 frozenset({8, 9}): array([-22.,   1.]),
 frozenset({8, 10}): array([-18.,   1.]),
 frozenset({8, 11}): array([-21.,   1.]),
 frozenset({8, 12}): array([-23.,   1.]),
 frozenset({8, 13}): array([ 1., -1.]),
 frozenset({8, 14}): array([ 1., -1.]),
 frozenset({9, 20}): array([ 1., -1.]),
 frozenset({9, 24}): array([  1., -22.]),
 frozenset({9, 25}): array([  1., -22.]),
 frozenset({9, 26}): array([  1., -22.]),
 frozenset({12, 15}): array([ 1., -5.]),
 frozenset({12, 16}): array([ 1., -5.]),
 frozenset({12, 17}): array([ 1., -5.]),
 frozenset({12, 18}): array([  1., -12.]),
 frozenset({12, 19}): array([  1., -12.]),
 frozenset({12, 20}): array([ 1., -1.]),
 frozenset({12, 21}): array([  1., -23.]),
 frozenset(

###con la misma cantidad de lineas inferimos, ahora con los nuevos invariantes

In [41]:
pipeline.get_anomalies(para)

log template path is /home/us1/git/loglizer/dayco_log.log_templates.csv
the mined 26 invariants are: {frozenset({4, 5}): array([-1.,  1.]), frozenset({10, 4}): array([-12.,   1.]), frozenset({11, 4}): array([-8.,  1.]), frozenset({4, 20}): array([-11.,   1.]), frozenset({24, 4}): array([ 1., -6.]), frozenset({25, 4}): array([ 1., -6.]), frozenset({26, 4}): array([ 1., -6.]), frozenset({8, 9}): array([-22.,   1.]), frozenset({8, 10}): array([-18.,   1.]), frozenset({8, 11}): array([-21.,   1.]), frozenset({8, 12}): array([-23.,   1.]), frozenset({8, 13}): array([ 1., -1.]), frozenset({8, 14}): array([ 1., -1.]), frozenset({9, 20}): array([ 1., -1.]), frozenset({24, 9}): array([  1., -22.]), frozenset({9, 25}): array([  1., -22.]), frozenset({9, 26}): array([  1., -22.]), frozenset({12, 15}): array([ 1., -5.]), frozenset({16, 12}): array([ 1., -5.]), frozenset({17, 12}): array([ 1., -5.]), frozenset({18, 12}): array([  1., -12.]), frozenset({19, 12}): array([  1., -12.]), frozenset({12, 

([0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
  0,
  1,
  1,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,


In [42]:
print(len(pipeline.anomalies))
for row_anomaly in pipeline.anomalies:
  print('##########################row anomalies')
  for event_anomaly in row_anomaly.anomaly_events:
    print('\n')
    print(event_anomaly.event_template)
    print(event_anomaly.row_index, event_anomaly.row)
    print(event_anomaly.invar_cols)
    print(event_anomaly.invariants)
    print(event_anomaly.start_window_index,event_anomaly.end_window_index)
    print(event_anomaly.matching_log_list)
    print(event_anomaly.log_lines_list)

10
##########################row anomalies


neighbor <*> Down BGP Notification <*>
43 [0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.]
[10, 4]
[-12.0, 1.0]
13 15
[]
[]


EIGRP-IPv4 2016: Neighbor 172.30.2.1 (GigabitEthernet3/8.3020) is down: holding time expired
43 [0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.]
[10, 4]
[-12.0, 1.0]
13 15
['14  Oct  5  10:23:39  200.74.216.13  320:  000736:  Oct  5  10:24:42.812  CCS:  %DUAL-5-NBRCHANGE:  EIGRP-IPv4 2016: Neighbor 172.30.2.1 (GigabitE...  c0779844  EIGRP-IPv4 2016: Neighbor 172.30.2.1 (GigabitE...']
['14']
##########################row anomalies


neighbor <*> Down BGP Notification <*>
61 [0. 0. 0. 0. 2. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.]
[10, 4]
[-12.0, 1.0]
23 27
[]
[]


EIGRP-IPv4 2016: Neighbor 172.30.2.1 (GigabitEthernet3/8.3020) is down: holding time expired
61 [0. 0. 0. 0. 2. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [43]:
!cp {repo_path}/rocore2.log {repo_path}/dayco_log.log

!wc -l {repo_path}/dayco_log.log
!ls -sh {repo_path}/dayco_log.log

file = open(repo_path + '/dayco_log.log', 'r')
lines = file.readlines()
file.close()
file = open(repo_path +'/dayco_log.log', 'a')
file.write(lines[-1][:-1])
file.close()
file = open(repo_path + '/dayco_log.log', 'r')
lines = file.readlines()
file.close()

234 /home/us1/git/loglizer/dayco_log.log
36K /home/us1/git/loglizer/dayco_log.log


In [44]:
pipeline.deepslash(para)

old, new 151 235
Parsing file: /home/us1/git/loglizer/dayco_log.log
Processed 100.0% of log lines.
Parsing done. [Time taken: 0:00:00.054670]
The raw data shape is (235, 1) and label shape is (235, 1)
there are 356 instances (sliding windows) in this dataset

There are 29 log events
log template path is /home/us1/git/loglizer/dayco_log.log_templates.csv
the mined 26 invariants are: {frozenset({4, 5}): array([-1.,  1.]), frozenset({10, 4}): array([-12.,   1.]), frozenset({11, 4}): array([-8.,  1.]), frozenset({4, 20}): array([-11.,   1.]), frozenset({24, 4}): array([ 1., -6.]), frozenset({25, 4}): array([ 1., -6.]), frozenset({26, 4}): array([ 1., -6.]), frozenset({8, 9}): array([-22.,   1.]), frozenset({8, 10}): array([-18.,   1.]), frozenset({8, 11}): array([-21.,   1.]), frozenset({8, 12}): array([-23.,   1.]), frozenset({8, 13}): array([ 1., -1.]), frozenset({8, 14}): array([ 1., -1.]), frozenset({9, 20}): array([ 1., -1.]), frozenset({24, 9}): array([  1., -22.]), frozenset({9, 25}

130  Oct  11  22:16:28  200.74.216.13  436:  001133:  Oct  11  22:17:37.021  CCS:  %DUAL-5-NBRCHANGE:  EIGRP-IPv4 2016: Neighbor 172.30.2.1 (GigabitE...  c0779844  EIGRP-IPv4 2016: Neighbor 172.30.2.1 (GigabitE...
*******8anomaly*****************
index, row 249 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 0.]
cols [10, 4] [-12.0, 1.0]

 invariant [10, 4]
>>> neighbor <*> Down BGP Notification <*>
start, end 149 156
len de eventos en window 1
152  Oct  14  01:08:41  200.74.216.13  458:  001254:  Oct  14  01:09:49.502  CCS:  %BGP-5-ADJCHANGE:  neighbor 200.16.71.1 Down BGP Notification sent  65c8b8ad  neighbor <*> Down BGP Notification <*>
>>> EIGRP-IPv4 2016: Neighbor 172.30.2.1 (GigabitEthernet3/8.3020) is down: holding time expired
start, end 149 156
len de eventos en window 0
*******8anomaly*****************
index, row 290 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0.]
cols [10, 4] [-12.0, 1.0]

 inva

In [45]:
print(len(pipeline.anomalies))
for row_anomaly in pipeline.anomalies:
  print('##########################row anomalies')
  for event_anomaly in row_anomaly.anomaly_events:
    print('\n')
    print(event_anomaly.event_template)
    print(event_anomaly.row_index, event_anomaly.row)
    print(event_anomaly.invar_cols)
    print(event_anomaly.invariants)
    print(event_anomaly.start_window_index,event_anomaly.end_window_index)
    print(event_anomaly.matching_log_list)
    print(event_anomaly.log_lines_list)

14
##########################row anomalies


neighbor <*> Down BGP Notification <*>
43 [0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0.]
[10, 4]
[-12.0, 1.0]
13 15
[]
[]


EIGRP-IPv4 2016: Neighbor 172.30.2.1 (GigabitEthernet3/8.3020) is down: holding time expired
43 [0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0.]
[10, 4]
[-12.0, 1.0]
13 15
['14  Oct  5  10:23:39  200.74.216.13  320:  000736:  Oct  5  10:24:42.812  CCS:  %DUAL-5-NBRCHANGE:  EIGRP-IPv4 2016: Neighbor 172.30.2.1 (GigabitE...  c0779844  EIGRP-IPv4 2016: Neighbor 172.30.2.1 (GigabitE...']
['14']
##########################row anomalies


neighbor <*> Down BGP Notification <*>
61 [0. 0. 0. 0. 2. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0.]
[10, 4]
[-12.0, 1.0]
23 27
[]
[]


EIGRP-IPv4 2016: Neighbor 172.30.2.1 (GigabitEthernet3/8.3020) is down: holding time expired
61 [0. 0. 0. 0. 2. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [46]:
pipeline.find_invariants(para)
print(len(pipeline.invar_dict))
pipeline.invar_dict

(356, 29)
SVD decomposition results (U,S,V) size:  (29, 29) (29,) (29, 29)
The size of invariants should be 28
the remaining features are:  [[0], [1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [12], [13], [14], [15], [16], [17], [18], [19], [20], [21], [22], [23], [24], [25], [26], [27], [28]]
selected matrix columns are [0, 1]
selected matrix columns are [0, 2]
selected matrix columns are [0, 3]
0 exits and discard the following vector: 
selected matrix columns are [0, 4]
0 exits and discard the following vector: 
selected matrix columns are [0, 5]
0 exits and discard the following vector: 
selected matrix columns are [0, 6]
selected matrix columns are [0, 7]
0 exits and discard the following vector: 
selected matrix columns are [0, 8]
selected matrix columns are [0, 9]
selected matrix columns are [0, 10]
selected matrix columns are [0, 11]
selected matrix columns are [0, 12]
selected matrix columns are [0, 13]
selected matrix columns are [0, 14]
selected matrix columns are 

{frozenset({4, 5}): array([-1.,  1.]),
 frozenset({4, 20}): array([-5.,  1.]),
 frozenset({4, 21}): array([ 1., -7.]),
 frozenset({4, 22}): array([ 1., -7.]),
 frozenset({4, 23}): array([ 1., -7.]),
 frozenset({4, 24}): array([ 1., -8.]),
 frozenset({4, 25}): array([ 1., -4.]),
 frozenset({4, 26}): array([ 1., -8.]),
 frozenset({8, 9}): array([-23.,   1.]),
 frozenset({8, 10}): array([-23.,   1.]),
 frozenset({8, 11}): array([-26.,   1.]),
 frozenset({8, 13}): array([-2.,  1.]),
 frozenset({8, 14}): array([-2.,  1.]),
 frozenset({9, 12}): array([-1.,  1.]),
 frozenset({9, 20}): array([ 1., -1.]),
 frozenset({9, 21}): array([  1., -22.]),
 frozenset({9, 22}): array([  1., -22.]),
 frozenset({9, 23}): array([  1., -22.]),
 frozenset({9, 24}): array([  1., -23.]),
 frozenset({9, 25}): array([  1., -11.]),
 frozenset({9, 26}): array([  1., -23.]),
 frozenset({9, 27}): array([  1., -23.]),
 frozenset({10, 12}): array([-1.,  1.]),
 frozenset({10, 20}): array([ 1., -1.]),
 frozenset({10, 21})

In [47]:
pipeline.event_count_matrix.shape

(356, 29)

In [48]:
pipeline.deepslash(para)
print(len(pipeline.anomalies))
for row_anomaly in pipeline.anomalies:
  print('##########################row anomalies')
  for event_anomaly in row_anomaly.anomaly_events:
    print('\n')
    print(event_anomaly.event_template)
    print(event_anomaly.row_index, event_anomaly.row)
    print(event_anomaly.invar_cols)
    print(event_anomaly.invariants)
    print(event_anomaly.start_window_index,event_anomaly.end_window_index)
    print(event_anomaly.matching_log_list)
    print(event_anomaly.log_lines_list)

old, new 235 235
14
##########################row anomalies


neighbor <*> Down BGP Notification <*>
43 [0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0.]
[10, 4]
[-12.0, 1.0]
13 15
[]
[]


EIGRP-IPv4 2016: Neighbor 172.30.2.1 (GigabitEthernet3/8.3020) is down: holding time expired
43 [0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0.]
[10, 4]
[-12.0, 1.0]
13 15
['14  Oct  5  10:23:39  200.74.216.13  320:  000736:  Oct  5  10:24:42.812  CCS:  %DUAL-5-NBRCHANGE:  EIGRP-IPv4 2016: Neighbor 172.30.2.1 (GigabitE...  c0779844  EIGRP-IPv4 2016: Neighbor 172.30.2.1 (GigabitE...']
['14']
##########################row anomalies


neighbor <*> Down BGP Notification <*>
61 [0. 0. 0. 0. 2. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0.]
[10, 4]
[-12.0, 1.0]
23 27
[]
[]


EIGRP-IPv4 2016: Neighbor 172.30.2.1 (GigabitEthernet3/8.3020) is down: holding time expired
61 [0. 0. 0. 0. 2. 2. 0. 0. 0. 0. 

### visualizamos los archivos relevantes de este experimento

In [49]:
!head {repo_path}/dayco_log.log 
!head {repo_path}/dayco_log.log_structured.csv  
!head {repo_path}/dayco_log.logTemplateMap.csv
!head {repo_path}/dayco_log.log_templates.csv

Oct  3 15:44:56 200.74.216.13 307: 000646: Oct  3 15:45:58.187 CCS: %SEC_LOGIN-5-LOGIN_SUCCESS: Login Success [user: admer] [Source: 200.74.215.2] [localport: 22] at 15:45:58 CCS Wed Oct 3 2018
Oct  3 16:15:00 200.74.216.13 308: 000647: Oct  3 16:16:01.919 CCS: %SYS-6-TTY_EXPIRE_TIMER: (exec timer expired, tty 1 (200.74.215.2)), user admer
Oct  3 16:15:00 200.74.216.13 309: 000648: Oct  3 16:16:01.919 CCS: %SYS-6-LOGOUT: User admer has exited tty session 1(200.74.215.2)
Oct  3 21:02:00 200.74.216.13 310: 000656: Oct  3 21:03:02.610 CCS: %SEC_LOGIN-5-LOGIN_SUCCESS: Login Success [user: admcod] [Source: 200.74.215.2] [localport: 22] at 21:03:02 CCS Wed Oct 3 2018
Oct  3 21:02:03 200.74.216.13 311: 000657: Oct  3 21:03:05.102 CCS: %SYS-6-LOGOUT: User admcod has exited tty session 1(200.74.215.2)
Oct  4 10:34:22 200.74.216.13 312: 000680: Oct  4 10:35:25.245 CCS: %SEC_LOGIN-5-LOGIN_SUCCESS: Login Success [user: admer] [Source: 200.74.215.2] [localport: 22] at 10:35:25 CCS Thu Oct 4 2018
Oc

In [50]:
from datetime import datetime, timedelta

In [51]:
def run_online(online_period, invariant_window, inference_window):
  online_period = timedelta(seconds=online_period)
  invar_window = timedelta(seconds=invariant_window)
  inference_window = timedelta(seconds=inference_window)
  start_time = datetime.now()
  last_invar_t = las_inference_t = now = datetime.now()
  while now - start_time < online_period:
    if now - last_invar_t > invar_window:
      last_invar_t = now
      print('calcular invar', now)
    if now - las_inference_t > inference_window:
      las_inference_t = now
      print('hacer inferencia', now, '<<')
    #print(now)
    now = datetime.now()

run_online(20, 4, 1)

hacer inferencia 2018-12-01 17:54:16.019266 <<
hacer inferencia 2018-12-01 17:54:17.019267 <<
hacer inferencia 2018-12-01 17:54:18.019268 <<
calcular invar 2018-12-01 17:54:19.019266
hacer inferencia 2018-12-01 17:54:19.019411 <<
hacer inferencia 2018-12-01 17:54:20.019412 <<
hacer inferencia 2018-12-01 17:54:21.019413 <<
hacer inferencia 2018-12-01 17:54:22.019414 <<
calcular invar 2018-12-01 17:54:23.019267
hacer inferencia 2018-12-01 17:54:23.019432 <<
hacer inferencia 2018-12-01 17:54:24.019433 <<
hacer inferencia 2018-12-01 17:54:25.019434 <<
hacer inferencia 2018-12-01 17:54:26.019435 <<
calcular invar 2018-12-01 17:54:27.019268
hacer inferencia 2018-12-01 17:54:27.019436 <<
hacer inferencia 2018-12-01 17:54:28.019437 <<
hacer inferencia 2018-12-01 17:54:29.019440 <<
hacer inferencia 2018-12-01 17:54:30.019441 <<
calcular invar 2018-12-01 17:54:31.019269
hacer inferencia 2018-12-01 17:54:31.019527 <<
hacer inferencia 2018-12-01 17:54:32.019528 <<
hacer inferencia 2018-12-01 17:54